# Interoperability between XProj and 3rd-party Xarray geospatial extensions

3rd-party geospatial extensions may leverage XProj in different ways:

- simply consume the API exposed via the `.proj` Dataset / DataArray accessor
- register a custom Xarray Dataset / DataArray accessor that implements one or more methods of the CRS interface (example below)
- implement one or more methods of the CRS interface in custom Xarray indexes (example below)

The CRS interface consists in a few specific "hook" methods called via XProj API.

In [1]:
import pyproj
import xarray as xr
import xproj

xr.set_options(display_expand_indexes=True);

## CRS-aware Xarray accessor

Here below is a basic example of a custom Xarray Dataset accessor that is also explictly registered as a "geo" accessor via the `xproj.register_geoaccessor` class decorator. Important note: the latter must be applied after (on top of) the Xarray register decorators.

Registering this "geo" accessor allows executing custom logic from within the accessor (via the CRS interface) when calling `xproj` API.

In [2]:
@xproj.register_geoaccessor
@xr.register_dataset_accessor("geo")
class GeoAccessor:

    def __init__(self, obj):
        self._obj = obj

    @property
    def crs(self):
        # Just reusing XProj's API
        # (Assuming this accessor only supports single-CRS datasets)
        return self._obj.proj.crs

    def __proj_set_crs__(self, crs_coord_name, crs):
        # Nothing much done here, just printing something before
        # returning the Xarray dataset unchanged

        print(f"from GeoAccessor: new CRS of {crs_coord_name!r} is {crs}!")
        return self._obj


Let's see if it works as expected.

In [3]:
# create an empty dataset, The `.geo.crs` property is uninitialized

ds = xr.Dataset()

ds.geo.crs is None

True

In [4]:
# initialize the CRS via `.proj.assign_crs()`

ds_wgs84 = ds.proj.assign_crs(spatial_ref=pyproj.CRS.from_user_input("epsg:4326"))

from GeoAccessor: new CRS of 'spatial_ref' is epsg:4326!


In [5]:
ds_wgs84

<xarray.Dataset> Size: 8B
Dimensions:      ()
Coordinates:
  * spatial_ref  int64 8B 0
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)

In [6]:
# Access CRS via the `.geo` accessor

ds_wgs84.geo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
ds_wgs84.xindexes.group_by_index()

[(CRSIndex
  <Geographic 2D CRS: EPSG:4326>
  Name: WGS 84
  Axis Info [ellipsoidal]:
  - Lat[north]: Geodetic latitude (degree)
  - Lon[east]: Geodetic longitude (degree)
  Area of Use:
  - name: World.
  - bounds: (-180.0, -90.0, 180.0, 90.0)
  Datum: World Geodetic System 1984 ensemble
  - Ellipsoid: WGS 84
  - Prime Meridian: Greenwich,
  {'spatial_ref': <xarray.Variable ()> Size: 8B
   array(0)})]

## CRS-aware Xarray index

Here below is a basic example of a custom Xarray index that adds some CRS-aware functionality on top of Xarray's default `PandasIndex`.

In [8]:
import warnings


class GeoIndex(xr.indexes.PandasIndex):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._crs = None

    def sel(self, *args, **kwargs):
        if self._crs is not None:
            warnings.warn(f"make sure that indexer labels have CRS {self._crs}!", UserWarning)
        
        return super().sel(*args, **kwargs)
    
    def __proj_set_crs__(self, crs_coord_name, crs):
        # `crs_coord_name` not used here (assuming single-CRS dataset)

        print(f"set CRS of index {self!r} to crs={crs}!")

        self._crs = crs
        return self

    def _copy(self, deep=True, memo=None):
        # bug in PandasIndex? crs attribute not copied here
        obj = super()._copy(deep=deep, memo=memo)
        obj._crs = self._crs
        return obj

    def _repr_inline_(self, max_width=70):
        return f"{type(self).__name__} (crs={self._crs})"

    def __repr__(self):
        return f"{type(self).__name__} (crs={self._crs})"


Let's see how it works

In [9]:
# Create a new Dataset with a latitude coordinate and a (default) PandasIndex

ds = xr.Dataset({"lat": [1, 2, 3]})
ds

<xarray.Dataset> Size: 24B
Dimensions:  (lat: 3)
Coordinates:
  * lat      (lat) int64 24B 1 2 3
Data variables:
    *empty*

In [10]:
# Replace the PandasIndex with a GeoIndex (crs not yet initialized)

ds_geo = ds.drop_indexes(["lat"]).set_xindex("lat", GeoIndex)
ds_geo

<xarray.Dataset> Size: 24B
Dimensions:  (lat: 3)
Coordinates:
  * lat      (lat) int64 24B 1 2 3
Data variables:
    *empty*

In [11]:
# initialize the CRS via `.proj.assign_crs()`

ds_geo_wgs84 = ds_geo.proj.assign_crs(spatial_ref="epsg:4326")

set CRS of index GeoIndex (crs=None) to crs=epsg:4326!
from GeoAccessor: new CRS of 'spatial_ref' is epsg:4326!


In [12]:
# The index of the `lat` coordinate also has its CRS initialized!

ds_geo_wgs84

<xarray.Dataset> Size: 32B
Dimensions:      (lat: 3)
Coordinates:
  * spatial_ref  int64 8B 0
  * lat          (lat) int64 24B 1 2 3
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)

In [13]:
# "CRS-aware" data selection (just a warning emitted here)

ds_geo_wgs84.sel(lat=1)

/var/folders/xd/3ls911kd6_n2wphwwd74b1dc0000gn/T/ipykernel_397/1484697919.py:12: UserWarning: make sure that indexer labels have CRS epsg:4326!
  warnings.warn(f"make sure that indexer labels have CRS {self._crs}!", UserWarning)


<xarray.Dataset> Size: 16B
Dimensions:      ()
Coordinates:
  * spatial_ref  int64 8B 0
    lat          int64 8B 1
Data variables:
    *empty*
Indexes:
    spatial_ref  CRSIndex (crs=EPSG:4326)